In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/cosmic-object-scanner/model_from_stratch/yolo/

In [ ]:
from model_utils import *
from torch import optim
from constants import *
from model import *
from Data_utils import *

device = 'cuda'
# Creating the model from YOLOv3 class 
model = YOLOv3(in_channels=3,num_classes=3).to(device) 
  
# Defining the optimizer 
optimizer = optim.Adam(model.parameters(), lr = leanring_rate) 
  
# Defining the loss function 
loss_fn = YOLOLoss() 
  
# Defining the scaler for mixed precision training 
scaler = torch.cuda.amp.GradScaler() 
  
# Defining the train dataset 
train_dataset = Dataset( 
    csv_file="../../og/train.csv", 
    image_dir="../../og/images/", 
    label_dir="../../og/labels/", 
    grid_sizes=[19,38,76],
    anchors=ANCHORS, 
    transform=train_transform 
) 
  
# Defining the train data loader 
train_loader = torch.utils.data.DataLoader( 
    train_dataset, 
    batch_size = 32, 
    num_workers = 4, 
    shuffle = True, 
    pin_memory = True, 
)

val_dataset = Dataset( 
    csv_file="../../og/train.csv", 
    image_dir="../../og/images/", 
    label_dir="../../og/labels/", 
    grid_sizes=[19,38,76],
    anchors=ANCHORS, 
    transform=test_transform 
)

# Defining the train data loader 
val_loader = torch.utils.data.DataLoader( 
    train_dataset, 
    batch_size = 1, 
    num_workers = 1, 
    shuffle = False, 
    pin_memory = True, 
)
  
# Scaling the anchors 
scaled_anchors = ( 
    torch.tensor(ANCHORS) * 
    torch.tensor(s).unsqueeze(1).unsqueeze(1).repeat(1,3,2) 
).to(device) 
  
# Training the model 
train_losses = []
val_losses = []
for e in range(1, epochs+1): 
    print("Epoch:", e) 
    train_loss = training_loop(train_loader, model, optimizer, loss_fn, scaler, scaled_anchors) 
    val_loss = evaluate(val_loader, model, loss_fn, scaled_anchors)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    # Saving the model 
    if save_model: 
        save_checkpoint(model, optimizer, filename=f"checkpoint.pth.tar")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_losses, marker='o', color='b', linestyle='-', linewidth=1, label='Training loss')
plt.plot(epochs, val_losses, marker='o', color='r', linestyle='-', linewidth=1, label='Validation loss')

plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()